In [3]:
import nbformat
import re

In [4]:
#import libcst as cst
#import libcst.matchers as m

In [5]:
nb = nbformat.read("portfolio/fundamentals/009-linreg-learner-solutions.ipynb", as_version=nbformat.NO_CONVERT)

In [6]:
cells = []
for cell in nb['cells']:
    if cell['cell_type'] != 'code':
        cells.append(cell)
        continue
    if cell.source.startswith("# remove"):
        continue
    cells.append(cell)
nb['cells'] = cells
#    cell.source = strip_code(cell.source)

In [32]:
source = cells[-11].source
print(source)

# ... <<<EOF
learner = Learner(
    dls=dataloaders,
    model=model,
    loss_func=loss_func,
    opt_func=SGD,
    metrics=[mae]
)
# EOF


In [39]:
def strip_source(source):
    result = []
    lines = source.split('\n')
    skip_next = False
    skip_until = None
    for i in range(len(lines)):
        if skip_next:
            skip_next = False
            continue
        line = lines[i]
        if skip_until is not None:
            if skip_until in line:
                # all done
                skip_until = None
            continue
        if i == len(lines) - 1:
            # last time unconditional
            result.append(line)
            continue
        is_commented = line.strip().startswith("# ")
        if is_commented:
            uncommented = re.sub(r'(\s*)(# )', r'\1', line)

            heredoc_match = re.match(r'(.+)\s+<<<(.+)', uncommented)
            if heredoc_match is not None:
                print(heredoc_match.group(1))
                end_str = heredoc_match.group(2)
                skip_until = end_str
                continue

            if '...' in line:
                # probable blank-filler
                match_re = '^' + re.escape(uncommented).replace(r'\.\.\.', '(.+)') + "$"
                if re.match(match_re, lines[i+1]):
                    skip_next = True
                    result.append(uncommented)
                    continue
                else:
                    print("WARNING: failed match", uncommented)
        result.append(line)
    return '\n'.join(result)
strip_source(source)

...


''

In [34]:
line

'# EOF'

In [16]:
re.sub(r'(\s*)(# )', r'\1', line)

'batch = ...'

In [15]:
re.escape(line).replace(r'\.\.\.', '(.+)')

'\\#\\ batch\\ =\\ (.+)'

In [46]:
class RefRemover(ast.NodeTransformer):
    def visit_Call(self, node):
        if getattr(node.func, 'id', '') == 'REFERENCE':
            return node.args[0]
        return node
    
    def visit_With(self, node):
        if node.items[0].context_expr.func.id == "REFBLOCK":
            return ast.Expr(value=ast.Constant("your code here"))
        return node

In [47]:
for cell in cells:
    if cell['cell_type'] != 'code':
        continue
    tree = ast.parse(cell.source)
    RefRemover().visit(tree)
    cell.source = astor.to_source(tree)
    

In [48]:
#nbformat.write(nb, "portfolio/fundamentals/009-linreg-learner.ipynb")